In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from flask import Flask, request, jsonify, render_template
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Load data
train_data = pd.read_csv('/kaggle/input/yahoomovies/ydata-ymovies-user-movie-ratings-train-v1_0.txt', sep='\t', header=None)
#movies_data = pd.read_csv('/kaggle/input/yahoomovies/ydata-ymovies-mapping-to-movielens-v1_0.txt', sep='\t', header=None)
with open('/kaggle/input/yahoomovies/ydata-ymovies-mapping-to-movielens-v1_0.txt', 'r', encoding='utf-8', errors='ignore') as f:
    content = f.readlines()

In [12]:
# Process the lines as needed, e.g., splitting by tabs or commas
data = []
for line in content:
    # Split line by tabs, commas, or other delimiters as appropriate
    values = line.strip().split('\t')  # Example: assuming tab-separated values
    data.append(values)

# Convert the list of lists into a DataFrame
movies_data = pd.DataFrame(data)

In [13]:
column_mapping = {0: 'user_id', 1:'movie_id',2: 'Rate',3:'Rating'}
train_data.rename(columns=column_mapping, inplace=True)
train_data.head()

,user_id,movie_id,Rate,Rating
0,1,1800029049,12,5
1,1,1804857429,8,4
2,1,1800030906,13,5
3,1,1800018548,11,5
4,1,1800256362,9,4


In [14]:
column_mapping = {0: 'movie_id', 1:'Title',2: 'movielens_movie_id'}
movies_data.rename(columns=column_mapping, inplace=True)
movies_data.head()

,movie_id,Title,movielens_movie_id
0,1800247298,Toy Story (1995),1
1,1800022746,Jumanji (1995),2
2,1800250021,Grumpier Old Men (1995),3
3,1800249828,Waiting to Exhale (1995),4
4,1800249488,Father of the Bride Part II (1995),5


In [15]:
user_ratings_df = train_data
movie_info_df = movies_data

In [16]:
movie_info_df

,movie_id,Title,movielens_movie_id
0,1800247298,Toy Story (1995),1
1,1800022746,Jumanji (1995),2
2,1800250021,Grumpier Old Men (1995),3
3,1800249828,Waiting to Exhale (1995),4
4,1800249488,Father of the Bride Part II (1995),5
...,...,...,...
3878,1802761123,Meet the Parents (2000),3948
3879,1800361191,Requiem for a Dream (2000),3949
3880,1802956884,Tigerland (2000),3950
3881,1800354250,Two Family House (2000),3951


In [17]:
user_ratings_df

,user_id,movie_id,Rate,Rating
0,1,1800029049,12,5
1,1,1804857429,8,4
2,1,1800030906,13,5
3,1,1800018548,11,5
4,1,1800256362,9,4
...,...,...,...,...
211226,7642,1808405417,11,5
211227,7642,1807839027,8,4
211228,7642,1808405428,11,5
211229,7642,1808429384,10,4


In [18]:
# from sklearn.neighbors import NearestNeighbors

# # Create pivot table for item-item based recommendation
# pivot_table = user_ratings_df.pivot_table(index='movie_id', columns='user_id', values='Rate', fill_value=0)

# # Instantiate and fit the k-nearest neighbors model
# knn = NearestNeighbors(metric='cosine', algorithm='brute')
# knn.fit(pivot_table)

# # Select a movie for which to make recommendations
# selected_movie_id = str(1800361191)
# selected_movie_index = movie_info_df[movie_info_df['movie_id'] == selected_movie_id].index[0]

# # Find the k-nearest neighbors to the selected movie
# k = 5  # Number of neighbors to consider
# distances, indices = knn.kneighbors(pivot_table.iloc[selected_movie_index].values.reshape(1, -1), n_neighbors=k+1)

# # Extract the movie IDs of the nearest neighbors (excluding the selected movie itself)
# nearest_movie_ids = pivot_table.index[indices.flatten()[1:]]
# print(nearest_movie_ids)
# # Get the movie information of the nearest neighbors from the movie_info_df DataFrame
# #nearest_movie_info = movie_info_df[movie_info_df['movie_id'].isin(nearest_movie_ids)]

# # Display the recommendations
# print("Recommendations for Movie ID:", nearest_movie_ids)


In [19]:
from sklearn.neighbors import NearestNeighbors

In [22]:
def get_movie_recommendations(movie_id, user_ratings_df, movie_info_df, k=5):
    """
    Get movie recommendations based on item-item collaborative filtering using k-nearest neighbors.
    
    Args:
        movie_id (str): Movie ID for which to get recommendations.
        user_ratings_df (pd.DataFrame): DataFrame containing user ratings data.
        movie_info_df (pd.DataFrame): DataFrame containing movie information data.
        k (int, optional): Number of neighbors to consider. Defaults to 5.
        
    Returns:
        list: List of movie IDs recommended for the given movie ID.
    """
    # Create pivot table for item-item based recommendation
    pivot_table = user_ratings_df.pivot_table(index='movie_id', columns='user_id', values='Rate', fill_value=0)

    # Instantiate and fit the k-nearest neighbors model
    knn = NearestNeighbors(metric='cosine', algorithm='brute')
    knn.fit(pivot_table)

    # Find the index of the selected movie in the pivot table
    selected_movie_index = movie_info_df[movie_info_df['movie_id'] == str(movie_id)].index[0]

    # Find the k-nearest neighbors to the selected movie
    distances, indices = knn.kneighbors(pivot_table.iloc[selected_movie_index].values.reshape(1, -1), n_neighbors=k+1)

    # Extract the movie IDs of the nearest neighbors (excluding the selected movie itself)
    nearest_movie_ids = pivot_table.index[indices.flatten()[1:]].tolist()

    return nearest_movie_ids

In [23]:
nearest_movie_ids=get_movie_recommendations(1800361191, user_ratings_df, movie_info_df, k=5)

In [ ]:
with open('/kaggle/input/yahoomovies/ydata-ymovies-movie-content-descr-v1_0.txt', 'r', encoding='utf-8', errors='ignore') as f:
    content = f.readlines()

In [ ]:
# Process the lines as needed, e.g., splitting by tabs or commas
data = []
for line in content:
    # Split line by tabs, commas, or other delimiters as appropriate
    values = line.strip().split('\t')  # Example: assuming tab-separated values
    data.append(values)

# Convert the list of lists into a DataFrame
df = pd.DataFrame(data)
column_mapping = {0: 'Yahoo! Movie ID', 1:'Title',2: 'Synopsis', 4:'Running Time', 5:'MPAA Rating',
                  6: 'Reasons for MPAA Rating', 
          7 :'Release Date',8: 'Distributor', 9:'Poster URL',10 :'Genres', 11:'Directors',12 :'Director IDs',
                  13: 'Crew Members', 
           14:'Crew IDs', 15:'Types of Crew', 16:'Actors',17 :'Actor IDs',18: 'Average Critic Rating',
                  19: 'Number of Critic Ratings', 
           20:'Number of Awards Won',21: 'Number of Awards Nominated', 22:'Awards Won',23: 'Awards Nominated',
                  24:'Rating from The Movie Mom', 
           25:'Review from The Movie Mom',26: 'Review Summaries',27: 'Anonymized Review Owners',
                  28:'Captions from Trailers/Clips', 
           29:"Greg's Preview URL", 30:"DVD Review URL", 31:'Global Non-Personalized Popularity (GNPP)',
                 32: 'Average User Rating', 
           33:'Number of Users Who Rated'}
df.rename(columns=column_mapping, inplace=True)
df = df.replace('\\N', '')

In [ ]:
for i in nearest_movie_ids:
    filtered_df=df[df['Yahoo! Movie ID'] == str(i)]
    print(filtered_df[['Yahoo! Movie ID','Title']])